In [1]:
# Check if the easydiffraction library is installed.
# If not, install it with the 'visualization' extras.
# Needed when running remotely (e.g. Colab) where the lib is absent.
import builtins
import importlib.util

if (hasattr(builtins, '__IPYTHON__') and
    importlib.util.find_spec('easydiffraction') is None):
    !pip install 'easydiffraction[visualization]'

# Pair Distribution Function: NaCl, XRD

This example demonstrates a pair distribution function (PDF) analysis
of NaCl, based on data collected from an X-ray powder diffraction
experiment.

The dataset is taken from:
https://github.com/diffpy/add2019-diffpy-cmi/tree/master

## Import Library

In [2]:
import easydiffraction as ed

## Create Project

In [3]:
project = ed.Project()

## Set Plotting Engine

In [4]:
# Keep the auto-selected engine. Alternatively, you can uncomment the
# line below to explicitly set the engine to the required one.
# project.plotter.engine = 'plotly'

In [5]:
# Set global plot range for plots
project.plotter.x_min = 2.0
project.plotter.x_max = 30.0

## Add Sample Model

In [6]:
project.sample_models.add(name='nacl')

In [7]:
project.sample_models['nacl'].space_group.name_h_m = 'F m -3 m'
project.sample_models['nacl'].space_group.it_coordinate_system_code = '1'
project.sample_models['nacl'].cell.length_a = 5.62
project.sample_models['nacl'].atom_sites.add(
    label='Na',
    type_symbol='Na',
    fract_x=0,
    fract_y=0,
    fract_z=0,
    wyckoff_letter='a',
    b_iso=1.0,
)
project.sample_models['nacl'].atom_sites.add(
    label='Cl',
    type_symbol='Cl',
    fract_x=0.5,
    fract_y=0.5,
    fract_z=0.5,
    wyckoff_letter='b',
    b_iso=1.0,
)

## Add Experiment

In [8]:
data_path = ed.download_data(id=4, destination='data')

Getting data...


Data #4: NaCl.gr


✅ Data #4 downloaded to 'data/ed-4.gr'


In [9]:
project.experiments.add(
    name='xray_pdf',
    data_path=data_path,
    sample_form='powder',
    beam_mode='constant wavelength',
    radiation_probe='xray',
    scattering_type='total',
)

Data loaded successfully


Experiment 🔬 'xray_pdf'. Number of data points: 5000


In [10]:
project.experiments['xray_pdf'].show_supported_peak_profile_types()

Supported peak profile types


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Peak profile type,Description
1,gaussian-damped-sinc,Gaussian-damped sinc profile for pair distribution function (PDF) analysis.


In [11]:
project.experiments['xray_pdf'].show_current_peak_profile_type()

Current peak profile type


PeakProfileTypeEnum.GAUSSIAN_DAMPED_SINC


In [12]:
project.experiments['xray_pdf'].peak_profile_type = 'gaussian-damped-sinc'

Peak profile type for experiment 'xray_pdf' changed to


gaussian-damped-sinc


In [13]:
project.experiments['xray_pdf'].peak.damp_q = 0.03
project.experiments['xray_pdf'].peak.broad_q = 0
project.experiments['xray_pdf'].peak.cutoff_q = 21
project.experiments['xray_pdf'].peak.sharp_delta_1 = 0
project.experiments['xray_pdf'].peak.sharp_delta_2 = 5
project.experiments['xray_pdf'].peak.damp_particle_diameter = 0

In [14]:
project.experiments['xray_pdf'].linked_phases.add(id='nacl', scale=0.5)

## Select Fitting Parameters

In [15]:
project.sample_models['nacl'].cell.length_a.free = True
project.sample_models['nacl'].atom_sites['Na'].b_iso.free = True
project.sample_models['nacl'].atom_sites['Cl'].b_iso.free = True

In [16]:
project.experiments['xray_pdf'].linked_phases['nacl'].scale.free = True
project.experiments['xray_pdf'].peak.damp_q.free = True
project.experiments['xray_pdf'].peak.sharp_delta_2.free = True

## Run Fitting

In [17]:
project.analysis.current_calculator = 'pdffit'
project.analysis.fit()
project.analysis.show_fit_results()

Current calculator changed to


pdffit


Using experiment 🔬 'xray_pdf' for 'single' fitting


🚀 Starting fit process with 'lmfit (leastsq)'...


📈 Goodness-of-fit (reduced χ²) change:


...

📏 Weighted R-factor (wR): 30.63%


📈 Fitted parameters:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,datablock,category,entry,parameter,start,fitted,uncertainty,units,change
1,nacl,cell,,length_a,5.6200,5.6175,N/A,Å,0.04 % ↓
2,nacl,atom_site,Na,b_iso,1.0000,1.0000,N/A,Å²,0.00 % ↓
3,nacl,atom_site,Cl,b_iso,1.0000,1.0000,N/A,Å²,0.00 % ↓
4,xray_pdf,linked_phases,nacl,scale,0.5000,0.4469,N/A,,10.61 % ↓
5,xray_pdf,peak,,damp_q,0.0300,0.0625,N/A,Å⁻¹,108.42 % ↑
6,xray_pdf,peak,,sharp_delta_2,5.0000,4.3628,N/A,Å²,12.74 % ↓


## Plot Measured vs Calculated

In [18]:
project.plot_meas_vs_calc(expt_name='xray_pdf')